# Data Transformation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import MinMaxScaler

%config IPCompleter.greedy=True

In [2]:

cat = pd.read_pickle('../../02_Data/03_Work/cat_result_quality.pickle')
num = pd.read_pickle('../../02_Data/03_Work/num_result_quality.pickle')


From previous phases we know that we have to:
(with pandas)
- Create target from state
- discretize months_since_last_delinquency
- in 'housing', join NONE and OTHER
- in 'purpose', join renewable_energy, educational and vacation.
(with scikit learn)
- we could work around 'description' with a TF-IDF Vectorizer
- convert num_derogatories to binary


## Creating variables and transformations with pandas

### Target for the PD model

In [11]:
cat.status.value_counts()

status
Fully Paid                                             66303
Current                                                54059
Charged Off                                            16411
Late (31-120 days)                                      1307
In Grace Period                                          533
Late (16-30 days)                                        270
Does not meet the credit policy. Status:Fully Paid       108
Does not meet the credit policy. Status:Charged Off       63
Default                                                    2
Name: count, dtype: int64

**Probability of Default** (PD) is the probability that a customer has to generate a default (delay of 90 days in the payment)

In [14]:
cat['target_pd'] = np.where(cat.status.isin(['Charged Off',
                                            'Does not meet the credit policy. Status:Charged Off',
                                            'Default']), 1, 0)
cat

,employment,employment_duration,verified_income,rating,housing,description,purpose,num_installments,status,target_pd
0,unknown,unknown,Verified,E,MORTGAGE,unknown,debt_consolidation,36 months,Fully Paid,0
1,Clark County Nevada,2 years,Source Verified,C,RENT,In the last couple of months the interest rate...,credit_card,36 months,Fully Paid,0
2,AVP Finance,7 years,Source Verified,B,MORTGAGE,unknown,debt_consolidation,60 months,Fully Paid,0
3,RN,2 years,Not Verified,C,RENT,unknown,debt_consolidation,36 months,Fully Paid,0
4,Hotel Leader,10+ years,Verified,C,RENT,unknown,debt_consolidation,36 months,Charged Off,1
...,...,...,...,...,...,...,...,...,...,...
139995,Office Manager,10+ years,Not Verified,C,MORTGAGE,unknown,debt_consolidation,36 months,Fully Paid,0
139996,Owner & President,6 years,Verified,B,MORTGAGE,unknown,other,36 months,Current,0
139997,Sr. Field Engineer,10+ years,Source Verified,D,MORTGAGE,unknown,small_business,36 months,Current,0
139998,Attorney,8 years,Source Verified,E,RENT,unknown,small_business,60 months,Charged Off,1


In [15]:
cat.drop(columns = 'status', inplace=True) 
# we won't need status anymore

### Target for the EAD model

Percentage from the principal that wasn't amortized (how much they still owe on the loan at the time of default). 

So: remaining/principal

In [16]:
num['remaining'] = num.principal - num.amount_amortized
num

,client_id,income,dti,num_mortgages,num_credit_lines,pct_cards_75p,pct_revolving_utilization,num_cancellations_12months,num_derogatories,months_since_last_delinquency,principal,interest_rate,installment_amount,amount_amortized,amount_recovered,remaining
0,59561505,36000.0,29.93,2.0,12.0,60.0,66.0,0.0,0.0,0.0,15100.0,18.25,547.80,15100.00,0.00,0.00
1,443000,59363.0,14.47,0.0,5.0,0.0,59.8,0.0,0.0,0.0,6000.0,13.22,202.81,6000.00,0.00,0.00
2,27640646,240000.0,7.74,5.0,16.0,20.0,34.1,0.0,0.0,0.0,24000.0,12.49,539.83,24000.00,0.00,0.00
3,31327120,61000.0,35.27,3.0,16.0,60.0,50.2,0.0,0.0,13.0,5600.0,14.65,193.17,5600.00,0.00,0.00
4,94241181,77000.0,16.95,2.0,14.0,60.0,49.4,0.0,0.0,30.0,9000.0,14.99,311.95,2563.02,0.00,6436.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139995,51876926,42000.0,20.85,6.0,9.0,100.0,81.6,0.0,0.0,13.0,8000.0,12.29,266.83,8000.00,0.00,0.00
139996,121031962,111697.0,16.63,2.0,10.0,50.0,26.3,0.0,1.0,6.0,10000.0,9.44,320.05,4388.51,0.00,5611.49
139997,135641397,285000.0,6.02,3.0,9.0,25.0,21.1,0.0,0.0,25.0,30000.0,17.47,1076.62,5387.53,0.00,24612.47
139998,53664762,168000.0,4.69,0.0,8.0,0.0,52.4,0.0,2.0,0.0,30050.0,18.25,767.17,2964.44,2750.88,27085.56


In [17]:
num['target_ead'] = num.remaining / num.principal
num

,client_id,income,dti,num_mortgages,num_credit_lines,pct_cards_75p,pct_revolving_utilization,num_cancellations_12months,num_derogatories,months_since_last_delinquency,principal,interest_rate,installment_amount,amount_amortized,amount_recovered,remaining,target_ead
0,59561505,36000.0,29.93,2.0,12.0,60.0,66.0,0.0,0.0,0.0,15100.0,18.25,547.80,15100.00,0.00,0.00,0.000000
1,443000,59363.0,14.47,0.0,5.0,0.0,59.8,0.0,0.0,0.0,6000.0,13.22,202.81,6000.00,0.00,0.00,0.000000
2,27640646,240000.0,7.74,5.0,16.0,20.0,34.1,0.0,0.0,0.0,24000.0,12.49,539.83,24000.00,0.00,0.00,0.000000
3,31327120,61000.0,35.27,3.0,16.0,60.0,50.2,0.0,0.0,13.0,5600.0,14.65,193.17,5600.00,0.00,0.00,0.000000
4,94241181,77000.0,16.95,2.0,14.0,60.0,49.4,0.0,0.0,30.0,9000.0,14.99,311.95,2563.02,0.00,6436.98,0.715220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139995,51876926,42000.0,20.85,6.0,9.0,100.0,81.6,0.0,0.0,13.0,8000.0,12.29,266.83,8000.00,0.00,0.00,0.000000
139996,121031962,111697.0,16.63,2.0,10.0,50.0,26.3,0.0,1.0,6.0,10000.0,9.44,320.05,4388.51,0.00,5611.49,0.561149
139997,135641397,285000.0,6.02,3.0,9.0,25.0,21.1,0.0,0.0,25.0,30000.0,17.47,1076.62,5387.53,0.00,24612.47,0.820416
139998,53664762,168000.0,4.69,0.0,8.0,0.0,52.4,0.0,2.0,0.0,30050.0,18.25,767.17,2964.44,2750.88,27085.56,0.901350


### Target for the LGD model

Lost Given Default

Percentage of the remaining that is NOT recovered if there's default.

In [18]:
num['target_lgd'] = 1 - (num.amount_recovered / num.remaining)
num

,client_id,income,dti,num_mortgages,num_credit_lines,pct_cards_75p,pct_revolving_utilization,num_cancellations_12months,num_derogatories,months_since_last_delinquency,principal,interest_rate,installment_amount,amount_amortized,amount_recovered,remaining,target_ead,target_lgd
0,59561505,36000.0,29.93,2.0,12.0,60.0,66.0,0.0,0.0,0.0,15100.0,18.25,547.80,15100.00,0.00,0.00,0.000000,NaN
1,443000,59363.0,14.47,0.0,5.0,0.0,59.8,0.0,0.0,0.0,6000.0,13.22,202.81,6000.00,0.00,0.00,0.000000,NaN
2,27640646,240000.0,7.74,5.0,16.0,20.0,34.1,0.0,0.0,0.0,24000.0,12.49,539.83,24000.00,0.00,0.00,0.000000,NaN
3,31327120,61000.0,35.27,3.0,16.0,60.0,50.2,0.0,0.0,13.0,5600.0,14.65,193.17,5600.00,0.00,0.00,0.000000,NaN
4,94241181,77000.0,16.95,2.0,14.0,60.0,49.4,0.0,0.0,30.0,9000.0,14.99,311.95,2563.02,0.00,6436.98,0.715220,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139995,51876926,42000.0,20.85,6.0,9.0,100.0,81.6,0.0,0.0,13.0,8000.0,12.29,266.83,8000.00,0.00,0.00,0.000000,NaN
139996,121031962,111697.0,16.63,2.0,10.0,50.0,26.3,0.0,1.0,6.0,10000.0,9.44,320.05,4388.51,0.00,5611.49,0.561149,1.000000
139997,135641397,285000.0,6.02,3.0,9.0,25.0,21.1,0.0,0.0,25.0,30000.0,17.47,1076.62,5387.53,0.00,24612.47,0.820416,1.000000
139998,53664762,168000.0,4.69,0.0,8.0,0.0,52.4,0.0,2.0,0.0,30050.0,18.25,767.17,2964.44,2750.88,27085.56,0.901350,0.898437


In [21]:
# there's nulls where remaining is 0, we can impute them for zeros

num['target_lgd'] = num['target_lgd'].fillna(0)
num

,client_id,income,dti,num_mortgages,num_credit_lines,pct_cards_75p,pct_revolving_utilization,num_cancellations_12months,num_derogatories,months_since_last_delinquency,principal,interest_rate,installment_amount,amount_amortized,amount_recovered,remaining,target_ead,target_lgd
0,59561505,36000.0,29.93,2.0,12.0,60.0,66.0,0.0,0.0,0.0,15100.0,18.25,547.80,15100.00,0.00,0.00,0.000000,0.000000
1,443000,59363.0,14.47,0.0,5.0,0.0,59.8,0.0,0.0,0.0,6000.0,13.22,202.81,6000.00,0.00,0.00,0.000000,0.000000
2,27640646,240000.0,7.74,5.0,16.0,20.0,34.1,0.0,0.0,0.0,24000.0,12.49,539.83,24000.00,0.00,0.00,0.000000,0.000000
3,31327120,61000.0,35.27,3.0,16.0,60.0,50.2,0.0,0.0,13.0,5600.0,14.65,193.17,5600.00,0.00,0.00,0.000000,0.000000
4,94241181,77000.0,16.95,2.0,14.0,60.0,49.4,0.0,0.0,30.0,9000.0,14.99,311.95,2563.02,0.00,6436.98,0.715220,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139995,51876926,42000.0,20.85,6.0,9.0,100.0,81.6,0.0,0.0,13.0,8000.0,12.29,266.83,8000.00,0.00,0.00,0.000000,0.000000
139996,121031962,111697.0,16.63,2.0,10.0,50.0,26.3,0.0,1.0,6.0,10000.0,9.44,320.05,4388.51,0.00,5611.49,0.561149,1.000000
139997,135641397,285000.0,6.02,3.0,9.0,25.0,21.1,0.0,0.0,25.0,30000.0,17.47,1076.62,5387.53,0.00,24612.47,0.820416,1.000000
139998,53664762,168000.0,4.69,0.0,8.0,0.0,52.4,0.0,2.0,0.0,30050.0,18.25,767.17,2964.44,2750.88,27085.56,0.901350,0.898437


In [22]:
cat['target_pd'].mean()

0.11848463928201587

In [23]:
num[['target_ead','target_lgd']].describe().T

,count,mean,std,min,25%,50%,75%,max
target_ead,139056.0,0.337315,0.369200,-0.000003,0.0,0.128457,0.725234,1.0
target_lgd,139056.0,0.508093,0.490401,-0.248392,0.0,0.848439,1.000000,1.0
